In [ ]:
//---- imports for HIPO4 library
import org.jlab.jnp.hipo4.io.*;
import org.jlab.jnp.hipo4.data.*;
//---- imports for GROOT library
import org.jlab.groot.data.*;
import org.jlab.groot.graphics.*;
//---- imports for PHYSICS library
import org.jlab.jnp.physics.*;
import org.jlab.jnp.reader.*;


HipoReader reader = new HipoReader(); // Create a reader obejct
//reader.open("/home/jnp/data/out_6489_2xx.hipo"); // open a file
reader.open("/Users/biselli/Data/clas12/out_6489_2xx.hipo"); // open a file

In [ ]:
Event     event = new Event();
Bank  particles = new Bank(reader.getSchemaFactory().getSchema("REC::Particle"));

In [ ]:
EventFilter  nDVCSFilter = new EventFilter("11:2112:22:Xn:X-");
EventFilter  pDVCSFilter = new EventFilter("11:2212:22:Xn:X-");
EventFilter  cohDVCSFilter = new EventFilter("11:45:22:Xn:X-");
EventFilter  elecFilter = new EventFilter("11:Xn:X+:X-");

In [ ]:
reader.getEvent(event,0); //Reads the first event and resets to the begining of the file
int times=0;
LorentzVector  vBeam   = new LorentzVector(0.0,0.0,10.2,10.2);
LorentzVector  vTarget = new LorentzVector(0.0,0.0,0.0,1.878);
LorentzVector  velectron = new LorentzVector();
LorentzVector  vtmp = new LorentzVector();
LorentzVector  vproton = new LorentzVector();
LorentzVector  vphoton = new LorentzVector();
LorentzVector  vdeuteron = new LorentzVector();
LorentzVector  vW = new LorentzVector();
LorentzVector  vQ2 = new LorentzVector();
LorentzVector  vMMass = new LorentzVector();
LorentzVector  vMMom = new LorentzVector();
LorentzVector  vinit = new LorentzVector();
vinit.copy(vBeam);
vinit.add(vTarget);
Particle  proton = new Particle();
Particle neutron = new Particle();
Particle deuteron = new Particle();
Particle electron = new Particle();
Particle photon = new Particle();
double Xbj;
double el_en_max=0;
double ph_en_max=0;
double d_en_max=0;
int nelec=0;
int ndvcs=0;
int nphot=0;
int ndeut=0;
int ncut=0;
int ne=-1;
int ng=-1;
int nd=-1;
double MNUC=1.878;
int PIDNUC=45;
double angleG=0;
System.out.println(vBeam.vect().x() + ", " + vBeam.vect().y() + ", " + vBeam.vect().z());
System.out.println(vMMom.vect().x() + ", " + vMMom.vect().y() + ", " + vMMom.vect().z());
System.out.println(vinit.vect().x() + ", " + vinit.vect().y() + ", " + vinit.vect().z());

In [ ]:
// histos
H1F W = new H1F("W" ,100, 0, 10.0);
W.setTitleX("W [GeV]");
H1F Q2 = new H1F("Q2",100, 0.1, 4.0);
Q2.setTitleX("Q^2 [GeV/c^2]");
H1F MMass = new H1F("MMass",100,-30,30);
MMass.setTitleX("Missing Mass Squared");
H1F MMom = new H1F("MMass",100,-30,30);
MMom.setTitleX("Missing Momentum");
H1F MMomx = new H1F("MMass",100,-10,10);
MMomx.setTitleX("Missing X Momentum");
H1F MMomy = new H1F("MMass",100,-10,10);
MMomy.setTitleX("Missing Y Momentum");
H1F MMomz = new H1F("MMass",100,-10,15);
MMomz.setTitleX("Missing Z Momentum");
H2F WvsQ2 = new H2F("W vs Q2", "Deuteron DVCS W vs Q2 before cuts", 100,0,7,100,0,10);
WvsQ2.setTitleX("W [GeV]");
WvsQ2.setTitleY("Q^2 [GeV/c^2]");
H2F ThvsPhi = new H2F("Theta vs Phi","Deuteron Theta vs Phi before cuts",100,-180,180,100,0,180);
ThvsPhi.setTitleX("Phi [Degrees]");
ThvsPhi.setTitleY("Theta [Degrees]");
H2F Q2vsXbj = new H2F("Q2 vs Xbj","Deuteron DVCS Q2 vs Xbj",100,0,10,100,0,1);
Q2vsXbj.setTitleX("Q^2 [GeV/c^2]");
Q2vsXbj.setTitleY("Xbj");
H2F MMvsMpz = new H2F("Missing Mass vs Missing Z Momentum","Missing Mass vs Missing Z Momentum",100,-2,2,100,-10,10);
MMvsMpz.setTitleX("Missing Mass");  
MMvsMpz.setTitleY("Missing Z Momentum");
H2F MpxvsMpz = new H2F("Missing X Momentum vs Missing Z Momentum","Missing X Momentum vs Missing Z Momentum",100,-2,2,100,-10,10);
MpxvsMpz.setTitleX("Missing X Momentum");
MpxvsMpz.setTitleY("Missing Z Momentum");


In [ ]:
//with loops
while(reader.hasNext()==true){
    reader.nextEvent(event);
    event.read(particles);
    el_en_max=0;
    ph_en_max=0;
    d_en_max=0;
    nelec=0;
    ndeut=0;
    nphot=0;
    if(particles.getRows()>3){
        for(int npart=0; npart<particles.getRows(); npart++){
        int pid = particles.getInt("pid", npart);
        if(pid==11){ 
            nelec++;
            vtmp.setPxPyPzM(particles.getFloat("px",npart),
                                 particles.getFloat("py",npart),
                                 particles.getFloat("pz",npart),
                                 0.0005);
            if(vtmp.e()>el_en_max)ne=npart;
        }
        else if(pid==22){
            nphot++;
            vtmp.setPxPyPzM(particles.getFloat("px",npart),
                                 particles.getFloat("py",npart),
                                 particles.getFloat("pz",npart),
                                 0.0);
            if(vtmp.e()>ph_en_max)ng=npart;
            
        }
        else if(pid==PIDNUC){
            ndeut++;
            vtmp.setPxPyPzM(particles.getFloat("px",npart),
                                 particles.getFloat("py",npart),
                                 particles.getFloat("pz",npart),
                                 MNUC);
            if(vtmp.e()>d_en_max)nd=npart;
            
        }   
    }
    if(ndeut>=1 && nelec>=1 && nphot>=1){
        velectron.setPxPyPzM(particles.getFloat("px",ne),
                                 particles.getFloat("py",ne),
                                 particles.getFloat("pz",ne),
                                 0.0005);
        vW.copy(vinit);
        vW.sub(velectron);
       
        vQ2.copy(vBeam);
        vQ2.sub(velectron);  
        vphoton.setPxPyPzM( particles.getFloat("px",ng),
                                particles.getFloat("py",ng),
                                particles.getFloat("pz",ng),
                                0.0);
        
        
        vdeuteron.setPxPyPzM(particles.getFloat("px",nd),
                                particles.getFloat("py",nd),
                                particles.getFloat("pz",nd),
                                MNUC);
    
        if(-vQ2.mass2()>1 && vW.mass()>2){
            ndvcs++;
            ThvsPhi.fill(vdeuteron.phi()*180./Math.PI,vdeuteron.theta()*180./Math.PI);
            Xbj = (-vQ2.mass2())/(2*0.938*(vBeam.p()-velectron.p()));
            Q2vsXbj.fill(-vQ2.mass2(),Xbj); 
            
            W.fill(vW.mass());
            Q2.fill(-vQ2.mass2());
            WvsQ2.fill(vW.mass(),-vQ2.mass2());
            vMMass.copy(vinit);
            vMMass.sub(velectron);
            vMMass.sub(vdeuteron);
                vMMass.sub(vphoton);
            MMass.fill(vMMass.mass2());
            
            //angleG = Math.acos((vphoton.vect().dot((vinit.sub(velectron).sub(vdeuteron)).vect())) / (vphoton.vect().mag() * (vinit.sub(velectron).sub(vdeuteron)).vect().mag()))*180./Math.PI;
            
           /* if(ndvcs%1000==0){
                //System.out.println(angleG);
                //System.out.println(vphoton.vect().theta());
                //System.out.println(vTarget.vect().theta());
                //System.out.println(vinit.sub(velectron).sub(vdeuteron).vect().theta());
                System.out.println(vBeam.vect().x() + ", " + vBeam.vect().y() + ", " + vBeam.vect().z());
                System.out.println(vinit.vect().x() + ", " + vinit.vect().y() + ", " + vinit.vect().z());
            }
            */
            //if(vMMass.mass2()>-1 && vMMass.mass2()<1 && (vphoton.theta()*180./Math.PI)<5){
            vMMom.copy(vinit);
            vMMom.sub(velectron);
            vMMom.sub(vdeuteron);
            vMMom.sub(vphoton);
            MMom.fill(vMMom.p());
            MMomx.fill(vMMom.px());
            MMomy.fill(vMMom.py());
            MMomz.fill(vMMom.pz());
            MMvsMpz.fill(vMMass.mass2(),vMMom.pz());
            MpxvsMpz.fill(vMMom.px(),vMMom.pz());
               // }
            }
        }
    }
                //if(ndvcs%1000==0){
                //System.out.println(angleG);
                //System.out.println(vphoton.vect().theta());
                //System.out.println(vTarget.vect().theta());
                //System.out.println(vinit.sub(velectron).sub(vdeuteron).vect().theta());
                System.out.println("vBeam "+vBeam.vect().x() + ", " + vBeam.vect().y() + ", " + vBeam.vect().z());
                System.out.println(vinit.vect().x() + ", " + vinit.vect().y() + ", " + vinit.vect().z());
            //}
}
    
System.out.println("total dvcs events: " + ndvcs);


In [ ]:
//with loops
EmbeddedCanvas ec = new EmbeddedCanvas(1200,1200);
ec.divide(2,2);
//ec.getPad(0).getAxisZ().setLog(true);
//ec.cd(0).draw(WvsQ2);
//ec.getPad(1).getAxisZ().setLog(true);
//ec.cd(1).draw(Q2vsXbj);
//ec.cd(2).draw(ThvsPhi);
ec.cd(0).draw(MMass);
ec.cd(1).draw(MMom);
ec.cd(2).draw(MMvsMpz);
ec.cd(3).draw(MpxvsMpz);
//ec.cd(0).draw(W);

ec.getScreenShot();


In [ ]:
//Gagik's way
while(reader.hasNext()==true){
    
    reader.nextEvent(event);
    event.read(particles);
    
    PhysicsEvent physEvent = DataManager.getPhysicsEvent(10.6,particles);
    if(elecFilter.isValid(physEvent)==true){
        nelec++;
        electron = physEvent.getParticleByPid(11,0);
        velectron.setPxPyPzM(electron.px(),electron.py(),electron.pz(),0.0005);
        //velectron.setPxPyPzM(particles.getFloat("px",0),particles.getFloat("py",0),particles.getFloat("pz",0),0.0005);
        vW.copy(vBeam);
        //vW.add(vTarget).sub(electron.vector());
        vW.add(vTarget).sub(velectron);
        W.fill(vW.mass());
        vQ2.copy(vBeam);
        //vQ2.sub(electron.vector());
        vQ2.sub(velectron);
        Q2.fill(-vQ2.mass2());
        WvsQ2.fill(vW.mass(),-vQ2.mass2());
        
        if(nDVCSFilter.isValid(physEvent)==true && -vQ2.mass2()>1 && vW.mass()>2){
            ndvcs++;
            proton = physEvent.getParticleByPid(2212,0);
        
            //Xbj = (-vQ2.mass2())/(2*proton.mass()*(vBeam.p()-electron.vector().p()));
            Xbj = (-vQ2.mass2())/(2*proton.mass()*(vBeam.p()-velectron.p()));
    
            //pXbj = (-vpQ2.mass2())/(2*proton.mass()*(vBeam.p()-electron.p()));
            //     pQ2vsXbj.fill(-vpQ2.mass2(),pXbj);
    

        
            Q2vsXbj.fill(-vQ2.mass2(),Xbj);   
    
            //neutron = physEvent.getParticleByPid(2112,0);
            //ThvsPhi.fill(neutron.phi()*180./Math.PI,neutron.theta()*180./Math.PI);

            //proton = physEvent.getParticleByPid(2212,0);
            //ThvsPhi.fill(proton.phi()*180./Math.PI,proton.theta()*180./Math.PI);

            //deuteron = physEvent.getParticleByPid(45,0);
            //ThvsPhi.fill(deuteron.phi()*180/Math.PI,deuteron.theta()*180./Math.PI);


    
            //times++;
            //System.out.println("loop has looped " + times + "times");
        
        }
    }
}
System.out.println("electrons  : " + nelec);
System.out.println("dvcs events: " + ndvcs);

In [ ]:
//Gagik's way
EmbeddedCanvas ec = new EmbeddedCanvas(800,400);
ec.divide(2,1);
ec.getPad(0).getAxisZ().setLog(true);
ec.cd(0).draw(WvsQ2);
ec.getPad(1).getAxisZ().setLog(true);
ec.cd(1).draw(Q2vsXbj);
//ec.cd(2).draw(ThvsPhi);
//ec.cd(3).draw();

ec.getScreenShot();


In [ ]:
LorentzVector  vBeam   = new LorentzVector(0.0,0.0,10.2,10.2);
LorentzVector  vTarget = new LorentzVector(0.0,0.0,0.0,1.878);

LorentzVector  vinit = new LorentzVector();
vinit.copy(vBeam);
vinit.add(vTarget);

System.out.println(vBeam.vect().x() + ", " + vBeam.vect().y() + ", " + vBeam.vect().z() + ", " +vBeam.e());
System.out.println(vinit.vect().x() + ", " + vinit.vect().y() + ", " + vinit.vect().z() + ", " +vinit.e());